In [2]:
import pandas as pd
from sqlalchemy import create_engine, inspect

user = 'root'
host = 'localhost'
port = 3306
database = 'makehaven'

engine = create_engine(f"mysql+pymysql://{user}@{host}:{port}/{database}")

In [3]:
contacts = pd.read_sql('select * from civicrm_contact', engine)
participants = pd.read_sql('select * from civicrm_participant', engine)
events = pd.read_sql('select * from civicrm_event', engine)

In [7]:
full_participants = pd.merge(
    participants,
    contacts,
    how='left',
    left_on='contact_id',
    right_on='external_identifier'
)
full_participants.head(10)

ValueError: You are trying to merge on int64 and object columns for key 'contact_id'. If you wish to proceed you should use pd.concat

In [18]:
all_signed_up_contacts = pd.read_sql("""
select cc.external_identifier as contact_id,
       display_name,
       events.id event_id,
       events.title,
       cp.status_id,
       status.label,
       fee_level
from civicrm_contact cc
inner join civicrm_participant cp on cc.external_identifier = cp.contact_id
inner join civicrm_event events on cp.event_id = events.id
left join civicrm_participant_status_type status on cp.status_id = status.id;
""", engine)



def get_participants_by_status_and_event_id(df, event_id, status_id):
    return df[(df["status_id"] == status_id) & (df["event_id"] == event_id)]




,contact_id,display_name,event_id,title,status_id,label,fee_level
36,1546,Ariel Bintang,19,Operation Opaque: Create Your Own Stained Glas...,1,Registered,Member Rate - 1
275,3673,Nicholas Beni,19,Operation Opaque: Create Your Own Stained Glas...,1,Registered,Standard Public Ticket - 1
287,3504,Michael Amendola,19,Operation Opaque: Create Your Own Stained Glas...,1,Registered,Member Rate - 1
1055,2766,James Oca,19,Operation Opaque: Create Your Own Stained Glas...,1,Registered,Member Rate - 1 (multiple participants)
1097,3458,Max Bruno,19,Operation Opaque: Create Your Own Stained Glas...,1,Registered,Standard Public Ticket - 1
